In [15]:
import numpy as np
import keras
from keras.models import load_model
model = load_model('model.h5')

In [263]:
model.layers[-1].activation = None
model.compile()

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 48, 48, 8)         224       
_________________________________________________________________
activation_8 (Activation)    (None, 48, 48, 8)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 48, 48, 8)         584       
_________________________________________________________________
activation_9 (Activation)    (None, 48, 48, 8)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 8)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 8)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 16)       

In [179]:
# weights = model.layers[-1].get_weights()[0]
# biases = model.layers[-1].get_weights()[1]
print(weights.shape)
print(biases.shape)

(256, 43)
(43,)


In [303]:
np.random.seed(1000)
def fc(x_l, x_u, w1, b1, w2):
    """
    [x: d neurons] -> [hidden: n neurons] -> [out: 1 neuron]
    x_l: (d,)
    x_u: (d,)
    w1: (n, d)
    b1: (n,)
    w2: (1, n)
    """
    d = x_l.shape[0]
    n = w1.shape[0]
    assert x_l.shape == x_u.shape == (d,)
    assert w1.shape == (n, d)
    assert b1.shape == (n,)
    assert w2.shape == (1, n)
    
    w1_pos = np.maximum(w1, 0)
    w1_neg = np.minimum(w1, 0)
    layer1_min = w1_pos.dot(x_l) + w1_neg.dot(x_u) + b1
    layer1_max = w1_pos.dot(x_u) + w1_neg.dot(x_l) + b1   
    
    # create diagonal array for unsure, positive, and negative neurons
    unsure = np.logical_and(layer1_min <= 0, layer1_max >= 0)
    positive = layer1_min >= 0
    negative = layer1_max <= 0
    
    unsure_diag = np.diag(unsure).astype(np.float32)
    pos_diag = np.diag(positive).astype(np.float32)
    neg_diag = np.diag(negative).astype(np.float32)
    
    assert(np.allclose(unsure_diag + pos_diag + neg_diag, np.eye(n)))
       
#     grad_lo_new = np.zeros(x_l.shape)
#     grad_hi_new = np.zeros(x_l.shape)
#     for i in range(d):
#         for k in range(n):
#             assert(w1[k, i] * w2[0, k] == prod_matrix[k, i])
#             if unsure[k]:
#                 grad_lo_new[i] += min(0, prod_matrix[k, i])
#                 grad_hi_new[i] += max(0, prod_matrix[k, i])
#             elif layer1_min[k] >= 0:
#                 grad_lo_new[i] += prod_matrix[k, i]
#                 grad_hi_new[i] += prod_matrix[k, i]
#     assert(np.allclose(grad_lo_new, grad_lo))
#     assert(np.allclose(grad_hi_new, grad_hi))

    prod_matrix = np.diag(w2[0]).dot(w1)
    prod_neg = np.minimum(0, prod_matrix)
    prod_pos = np.maximum(0, prod_matrix)
    grad_lo = np.sum(unsure_diag.dot(prod_neg) + pos_diag.dot(prod_matrix), axis = 0)
    grad_hi = np.sum(unsure_diag.dot(prod_pos) + pos_diag.dot(prod_matrix), axis = 0)

    return grad_lo, grad_hi
    

# def f(x):
#     return w2.dot(np.maximum(0, w1.dot(x) + b1))

# def grad(x):
#     x = x.astype(np.float32)
#     ans = np.zeros(x.shape)
#     for i in range(x.shape[0]):
#         x2 = np.copy(x)
#         eps = 0.01
#         x2[i] += eps
#         ans[i]= ((f(x2) - f(x)) / eps)
#     return ans

# x_l = np.array([-1, 5, 3])
# x_u = np.array([1, 8, 6])
# w1 = np.random.randint(-3, 3, (4, 3))
# b1 = np.zeros(4,)
# w2 = np.ones((1, 4))
# # print(x_l)
# # print(x_u)
# # print(w1)
# # print(b1)
# # print(w2)

# x_real = np.array([-1, 3, 4])
# gl, gh = fc(x_l, x_u, w1, b1, w2)
# # print(gl, gh)
# # gl, gh = fc(x_real-0.01, x_real+0.01, w1, b1, w2)
# def relu(x):
#     return max(0, x)
# def ff(a, b, c):
#     return relu(-3*b) + relu(a-2*b+2*c) +relu(-3*a-2*b+2*c) +relu(-3*a-2*b+c)


In [338]:


grad_model = tf.keras.models.Model(
    [model.inputs], [model.get_layer(index = -4).output, model.output]
)

w1 = model.layers[-2].get_weights()[0]
b1 = model.layers[-2].get_weights()[1]
w2 = model.layers[-1].get_weights()[0][:, target_class:target_class+1]
b2 = model.layers[-1].get_weights()[1][target_class]

# get true gradient of output with respect to convolutional layer outputs
with tf.GradientTape() as tape:
    inputs = tf.cast(x, tf.float64)
    conv_outputs, predictions = grad_model(x)
    loss = predictions[:, target_class]

grads = tape.gradient(loss, conv_outputs)
grads_flat = grads.numpy().flatten()

# toy example 
target_class = 13
np.random.seed(1000)
x = np.random.normal(0, 1, (1, 48, 48, 3))
x = conv_outputs.numpy().flatten()
grad_lo, grad_hi = fc(x - 0.01, x + 0.01, w1.T, b1, w2.T)
assert(np.sum(np.logical_and(grad_lo - eps <= grads_flat, grads_flat <= grad_hi + eps)) == 9216)

tf.Tensor([6.887344], shape=(1,), dtype=float32)
(9216,)
(9216,)


In [337]:
print(np.mean(grad_hi - grad_lo))


0.11159996
